In [ ]:
!pip install -r ../requirements.txt

# Configuration

In [ ]:
ROOT_DIR = "../"
DATA_DIR = ROOT_DIR + "data/"

DATASET_2016_10a_FILENAME = DATA_DIR + "RML2016.10a_dict.pkl"

NN_NAME = 'NEURAL_NETWORK_NAME'

DEFAULT_CONFIG_FILENAME = ROOT_DIR + "default.ini"

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read(DEFAULT_CONFIG_FILENAME)

NEURAL_NETWORK_NAME = config.get('NeuralNetworkNames', NN_NAME)
NEURAL_NETWORK_FILENAME = NEURAL_NETWORK_NAME + '.wts.h5'

TRAINING_SET_DIMENSION = config.getfloat('TrainingValues', 'TrainingSetSplitPercentage')
DROPOUT_RATE = config.getfloat('TrainingValues', 'DropoutRate')

# Set up some params 
NB_EPOCH = config.getint('TrainingValues', 'Epochs') # number of epochs to train on
BATCH_SIZE = config.getint('TrainingValues', 'BatchSize') # training batch size

# Imports

In [ ]:
import sys
sys.path.append(ROOT_DIR)

import neural_networks
import datasetlib
import traintestsplitlib
import trainlib
import evaluationlib
import dftlib

# Dataset

## Dataset in a pandas Dataframe

In [ ]:
dataset_df = datasetlib.load_dataset_dataframe(DATASET_2016_10a_FILENAME)

## Raw dataset

In [ ]:
dataset_df

# Neural network

## Configuration

In [ ]:
config = configparser.ConfigParser()
config.read(DEFAULT_CONFIG_FILENAME)

NEURAL_NETWORK_NAME = config.get('NeuralNetworkNames', NN_NAME)
NEURAL_NETWORK_FILENAME = NEURAL_NETWORK_NAME + '.wts.h5'

TRAINING_SET_DIMENSION = config.getfloat('TrainingValues', 'TrainingSetSplitPercentage')
DROPOUT_RATE = config.getfloat('TrainingValues', 'DropoutRate')

# Set up some params 
NB_EPOCH = config.getint('TrainingValues', 'Epochs') # number of epochs to train on
BATCH_SIZE = config.getint('TrainingValues', 'BatchSize') # training batch size

## Data prepare

In [ ]:
signals = datasetlib.signals(dataset_df)
labels = datasetlib.labels(dataset_df)

# [QPSQ, BPSK, ...]
mods = datasetlib.mods(dataset_df)
# [-18, -16, ...]
snrs = datasetlib.unique_snrs(dataset_df)

## Data transformation

In [ ]:
transformed_signals = dftlib.all_I_Q_to_DFT(signals)

### Training and Test data partitioning

In [ ]:
n_elements = len(signals)
training_set_percentage = TRAINING_SET_DIMENSION

train_index, test_index = traintestsplitlib.train_test_random_indexes(n_elements, training_set_percentage)

iq_train, transformed_train, iq_test, transformed_test, Y_train, Y_test = traintestsplitlib.split_double_x_y_train_test(signals, transformed_signals, labels, mods, train_index, test_index)

## Build the Neural Network

In [ ]:
iq_shp = list(iq_train.shape[1:])
transformed_shp = list(transformed_train.shape[1:])

print(iq_train.shape)
print(iq_shp)
print(transformed_train.shape)
print(transformed_shp)

classes = mods

In [ ]:
model = neural_networks.NEURAL_NETWORK(iq_shp, transformed_shp)

model.summary()

### Train the model

In [ ]:
# perform training ...
#   - call the main training loop in keras for our network+dataset
filepath = NEURAL_NETWORK_FILENAME

history, model = trainlib.train_double_input(filepath, model, iq_train, transformed_train, Y_train, BATCH_SIZE, NB_EPOCH)

# Evaluate and Plot Model Performance

In [ ]:
# Show simple version of performance
score = model.evaluate((iq_test, transformed_test), Y_test, batch_size=BATCH_SIZE)
print(score)

## Loss

In [ ]:
evaluationlib.show_loss_curves(history)

## Confusion matrix

In [ ]:
evaluationlib.plot_double_input_confusion_matrix(model, NEURAL_NETWORK_NAME, iq_test, transformed_test, Y_test, BATCH_SIZE, classes)

## Confusion matrix for each SNR

In [ ]:
acc = evaluationlib.plot_double_input_confusion_matrix_each_snr(model, NEURAL_NETWORK_NAME, snrs, dataset_df, iq_test, transformed_test, Y_test, test_index, classes)

## Accuracy performance

In [ ]:
evaluationlib.accuracy_dataframe(acc)

In [ ]:
# Save results to a pickle file for plotting later
# fd = open(OUTPUT_FILENAME,'wb')
# pickle.dump( ("CNN2", 0.5, acc) , fd )

In [ ]:
evaluationlib.accuracy_curve(snrs, acc, NEURAL_NETWORK_NAME)